In [ ]:
# import libraries.
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
import datetime
tf.enable_eager_execution()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import h5py
import glob
import matplotlib.pyplot as plt
import time
import os
import librosa
from librosa.core import stft, istft
import smtplib
from natsort import natsorted


### load rbm weights

In [ ]:

pre_rbm = 'pre_44'
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
# Data_path = os.getcwd()[0:-5]

result_path = os.path.normpath(os.path.join(Data_path,'results','models'))
l10=np.loadtxt(os.path.join(result_path,pre_rbm,'layer10.txt'))
l11=np.loadtxt(os.path.join(result_path,pre_rbm,'layer11.txt'))
l12=np.loadtxt(os.path.join(result_path,pre_rbm,'layer12.txt'))
l20=np.loadtxt(os.path.join(result_path,pre_rbm,'layer20.txt'))
l21=np.loadtxt(os.path.join(result_path,pre_rbm,'layer21.txt'))
l22=np.loadtxt(os.path.join(result_path,pre_rbm,'layer22.txt'))
l30=np.loadtxt(os.path.join(result_path,pre_rbm,'layer30.txt'))
l31=np.loadtxt(os.path.join(result_path,pre_rbm,'layer31.txt'))
l32=np.loadtxt(os.path.join(result_path,pre_rbm,'layer32.txt'))

### define variables and filenames


In [ ]:
Data_path = os.getcwd()[0:-5]
tfrecord_folder_parent = 'tfrecord_files'
tfrecord_folder = 'ftr_refrmd_30h_2_w7_norm'
tfrecord_val_folder = 'ftr_refrmd_dev_w7_norm_15h'
global ckpt_folder
ckpt_folder = '208'
global batch_size
batch_size = 128
len_data = (3337525, 257)
val_len = (700000,1799)
steps = len_data[0] // batch_size
val_steps = val_len[0] // batch_size
w=7
buffersize = 1800000
global datalen
epochs_num = 30
h = [2048,2048,1024]
seed = 7


# train model and define it

In [ ]:
Data_path = os.getcwd()[0:-5]
result_path = os.path.normpath(os.path.join(Data_path,'results'))

seed = 7
from tensorflow.keras.layers import Activation
np.random.seed(seed)
act = layers.LeakyReLU(alpha=0.1)
model = Sequential()
model.add(Dense(h[0], input_dim = w*len_data[1], kernel_initializer= tf.constant_initializer(l10), bias_initializer = tf.constant_initializer(l12)))
model.add(Activation('sigmoid'))
model.add(Dense(h[1], kernel_initializer= tf.constant_initializer(l20), bias_initializer = tf.constant_initializer(l22)))
model.add(Activation('sigmoid'))
model.add(Dense(h[2], kernel_initializer= tf.constant_initializer(l30), bias_initializer = tf.constant_initializer(l32)))
model.add(Activation('sigmoid'))
model.add(Dense(len_data[1]))


In [ ]:
model.summary()

### this section is used for tfrecord files

In [ ]:

def _parse_function(example_proto):
    features = {"X": tf.FixedLenFeature((w*257), tf.float32),
              "Y": tf.FixedLenFeature((257), tf.float32)}
    parsed_features = tf.parse_single_example(example_proto, features)
    return parsed_features["X"], parsed_features["Y"]

tfrecord_path = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
subset = 'filenum'
files = tf.io.matching_files(os.path.normpath(os.path.join(tfrecord_path, '%s*' % subset)))
shards = tf.data.Dataset.from_tensor_slices(files)
shards = shards.shuffle(tf.cast(tf.shape(files)[0], tf.int64))
dataset = shards.interleave(tf.data.TFRecordDataset, cycle_length=6)
dataset = dataset.shuffle(buffer_size=buffersize)
dataset = dataset.repeat(epochs_num)
dataset = dataset.map(_parse_function)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(batch_size)

tfrecord_path_val = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_val_folder))
subset = 'filenum'
files_val = tf.io.matching_files(os.path.normpath(os.path.join(tfrecord_path_val, '%s*' % subset)))
shards_val = tf.data.Dataset.from_tensor_slices(files_val)
dataset_val = tf.data.TFRecordDataset(shards_val)
dataset_val = dataset_val.repeat(epochs_num)
dataset_val = dataset_val.map(_parse_function)
dataset_val = dataset_val.batch(batch_size)
    


### define how we want the models to be saved

In [ ]:
files = glob.glob(os.path.join(result_path,"checkpoints",ckpt_folder,'*'))
for f in files:
    os.remove(f)
if not os.path.exists(os.path.join(result_path,"checkpoints",ckpt_folder)):
    os.makedirs(os.path.join(result_path,"checkpoints",ckpt_folder))
print(datetime.datetime.now())
checkpoint_path = os.path.normpath(os.path.join(result_path,"checkpoints",ckpt_folder,"weights.{epoch:02d}.hdf5"))
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True, save_best_only = True)

### define scheduler

In [ ]:
def lr_scheduler(epoch, lr):
    decay_rate = 0.9
    if epoch>10:
        return lr * decay_rate
    return lr

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

### define optimizer


In [ ]:
opt = optimizers.SGD(lr=0.1, momentum=0,nesterov = False)
model.compile(loss='mse', optimizer=opt)
history = model.fit( dataset, steps_per_epoch=steps,epochs=epochs_num,callbacks = [cp_callback, lr_callbacks], verbose=1,validation_data=dataset_val,validation_steps=val_steps)
model_json = model.to_json()
with open(os.path.normpath(os.path.join(result_path, 'models', "model_"+ckpt_folder+".json")), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.normpath(os.path.join(result_path, 'models', "model_"+ckpt_folder+".h5")))
print("Saved model to disk")

### save models

In [ ]:
model.save(os.path.normpath(os.path.join(result_path, 'models', "model_"+ckpt_folder+"_whole.h5")))

In [ ]:
model_json = model.to_json()
with open(os.path.normpath(os.path.join(result_path, 'models', "model_"+ckpt_folder+".json")), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.normpath(os.path.join(result_path, 'models', "model_"+ckpt_folder+".h5")))
print("Saved model to disk")
# h5f.close()   
print(datetime.datetime.now())
%matplotlib inline
# summarize history for loss
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='validation')
plt.legend()
# plt.legend(['valid'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')


plt.savefig(os.path.normpath(os.path.join(result_path,'images',ckpt_folder+'.png')))
plt.show()